## Imports & paths

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import plotting
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

# Paths
PROCESSED_PRICES = Path("data/processed/prices_adj.csv")
TASK2_TSLA_FORECAST = Path("outputs/task2_tsla_forecast.csv")   # expected: columns ['Date','Forecast']
FIG_PATH = Path("reports/figures/task4_efficient_frontier.png")
WEIGHTS_PATH = Path("outputs/task4_optimal_weights.csv")
PERF_PATH = Path("outputs/task4_portfolio_performance.txt")
EXPECTED_RETURNS_PATH = Path("outputs/task4_expected_returns.csv")

# Create output dirs
FIG_PATH.parent.mkdir(parents=True, exist_ok=True)
WEIGHTS_PATH.parent.mkdir(parents=True, exist_ok=True)


## Load prices & compute daily returns

In [ ]:
# Load prices and verify tickers
prices = pd.read_csv(PROCESSED_PRICES, parse_dates=["Date"], index_col="Date").sort_index()
tickers = ["TSLA", "BND", "SPY"]
assert all(t in prices.columns for t in tickers), "Missing required tickers in prices_adj.csv"

# Daily returns
returns = prices[tickers].pct_change().dropna()

# Annualize covariance via Ledoit-Wolf shrinkage (more stable)
S = CovarianceShrinkage(prices[tickers]).ledoit_wolf()


## Build expected returns vector (TSLA from forecast, BND/SPY from history)

In [ ]:
def annualize_from_daily(daily_returns: pd.Series) -> float:
    mu_daily = daily_returns.mean()
    return (1 + mu_daily) ** 252 - 1

# Historical expected annual returns for BND & SPY
exp_bnd = annualize_from_daily(returns["BND"])
exp_spy = annualize_from_daily(returns["SPY"])

# TSLA expected annual return from forecast (preferred) or fallback
use_fallback = False
if TASK2_TSLA_FORECAST.exists():
    fc = pd.read_csv(TASK2_TSLA_FORECAST, parse_dates=["Date"])
    fc = fc.sort_values("Date").set_index("Date")
    if "Forecast" not in fc.columns:
        raise ValueError("Task 2 forecast file must contain a 'Forecast' column.")
    # Forecasted daily returns from forecasted price path
    tsla_fc_daily = fc["Forecast"].pct_change().dropna()
    if tsla_fc_daily.empty:
        use_fallback = True
    else:
        exp_tsla = annualize_from_daily(tsla_fc_daily)
else:
    use_fallback = True

if use_fallback:
    # Fallback: last 252 trading days of TSLA historical data
    tsla_last_year = returns["TSLA"].dropna().iloc[-252:]
    exp_tsla = annualize_from_daily(tsla_last_year)
    print("TSLA forecast file not found or invalid; using last-year historical mean as fallback.")

# Expected returns vector
mu = pd.Series({"TSLA": exp_tsla, "BND": exp_bnd, "SPY": exp_spy})
mu.to_csv(EXPECTED_RETURNS_PATH)
print("Expected annual returns (from forecast + history):")
display(mu.to_frame("ExpectedAnnualReturn"))


## Efficient Frontier with custom expected returns

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)

# To plot a smooth frontier, we reconstruct it with many target returns
fig, ax = plt.subplots(figsize=(9, 6))
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.title("Forecast-Driven Efficient Frontier (TSLA from forecast; BND/SPY from history)")
plt.xlabel("Volatility (Risk)")
plt.ylabel("Expected Return")
plt.tight_layout()
plt.show()

fig.savefig(FIG_PATH, dpi=150)
print(f"Saved frontier plot to {FIG_PATH}")


## Identify & mark key portfolios: Max Sharpe and Min Volatility

In [ ]:
# Compute key portfolios
ef_max = EfficientFrontier(mu, S)
w_max_sharpe = ef_max.max_sharpe()
w_max_sharpe = ef_max.clean_weights()
perf_max = ef_max.portfolio_performance()  # (ret, vol, sharpe)

ef_min = EfficientFrontier(mu, S)
w_min_vol = ef_min.min_volatility()
w_min_vol = ef_min.clean_weights()
perf_min = ef_min.portfolio_performance()

# Plot with markers
fig, ax = plt.subplots(figsize=(9, 6))
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)

# Scatter the two portfolios
ret_max, vol_max, sharpe_max = perf_max
ret_min, vol_min, sharpe_min = perf_min

ax.scatter([vol_max], [ret_max], marker="*", s=250, label="Max Sharpe", zorder=5)
ax.scatter([vol_min], [ret_min], marker="o", s=200, label="Min Vol", zorder=5)

ax.legend()
ax.set_title("Efficient Frontier with Key Portfolios")
ax.set_xlabel("Volatility (Risk)")
ax.set_ylabel("Expected Return")
plt.tight_layout()
plt.show()

# Save plot
fig.savefig(FIG_PATH, dpi=150)
print(f"Updated frontier plot (with markers) saved to {FIG_PATH}")

# Save weights & performance
summary = {
    "MaxSharpe": {"weights": w_max_sharpe, "performance": {"return": ret_max, "vol": vol_max, "sharpe": sharpe_max}},
    "MinVol":    {"weights": w_min_vol,    "performance": {"return": ret_min, "vol": vol_min, "sharpe": sharpe_min}},
}
pd.Series(w_max_sharpe, name="MaxSharpe").to_csv(WEIGHTS_PATH)
with open(PERF_PATH, "w") as f:
    f.write("Max Sharpe Portfolio:\n")
    f.write(f"Expected Return: {ret_max:.4f}\nVolatility: {vol_max:.4f}\nSharpe Ratio: {sharpe_max:.4f}\n\n")
    f.write("Min Volatility Portfolio:\n")
    f.write(f"Expected Return: {ret_min:.4f}\nVolatility: {vol_min:.4f}\nSharpe Ratio: {sharpe_min:.4f}\n")

print("Saved weights and performance to outputs/")


## Choose and print a recommended portfolio

In [ ]:
# Choose recommendation policy:
RECOMMEND = "MaxSharpe"   # or "MinVol"

if RECOMMEND == "MaxSharpe":
    chosen_w = w_max_sharpe
    chosen_perf = perf_max
    rationale = "Prioritizing maximum risk-adjusted return (Sharpe)."
else:
    chosen_w = w_min_vol
    chosen_perf = perf_min
    rationale = "Prioritizing lower absolute risk (minimum volatility)."

ret_c, vol_c, sharpe_c = chosen_perf

print("=== Recommended Portfolio ===")
print(f"Policy: {RECOMMEND} — {rationale}")
print("Weights:")
for k, v in chosen_w.items():
    print(f"  {k}: {v:.4f}")

print("\nPerformance (annualized):")
print(f"  Expected Return: {ret_c:.4f}")
print(f"  Volatility:      {vol_c:.4f}")
print(f"  Sharpe Ratio:    {sharpe_c:.4f}")
